In [ ]:
##############################################################
#        Script: Tratamento do Arquivo do DataDesktop        #
#                    Developed By: Gus                       #
#        E-mail: gustavo.lima@cortex-intelligence.com        #
##############################################################


# Imports
import pandas as pd
from numpy import nan

# Necessário salvar o arquivo na pasta abaixo e alterar o nome da variavel \"arquivo\"
input_file_path = 'C:\\Projetos\\Python\\Data Desktop\\'
arquivo_input = 'PSNEW-3846' + '.xlsx'


# --------------------------------------------------------------------------------- #
# --------------------------Tratamento - Dados Cadastrais ------------------------- #
# --------------------------------------------------------------------------------- #


#df = pd.read_excel(input_file_path + arquivo_input)
df = pd.read_excel('PSNEW-3846.xlsx')


#Seleciona só as colunas necessárias pro output
df1 = df.filter(items=['Código CNPJ','Razão social','Nome fantasia','Logradouro', 'Numero','Complemento', 'Bairro', 'Cidade', 'UF', 'CEP', 'Porte', 'Capital Social até', 'Faturamento até','Quantidade de funcionários até', 'Data da abertura','Tipo de unidade', ' Tipo situação', 'Tipo Enquadramento Porte', 'Código atividade econômica primária', 'Nome atividade econômica primária', 'Código da natureza jurídica', 'Nome natureza jurídica geral', 'E-mail na Receita Federal','Tipo de classificação jurídica', 'Website1', 'Website2', 'Website3', 'Website4', 'Website5', 'Nome do sócio 1', 'Qualificação sócio 1','Tipo qualificação sócio 1', 'Nome do sócio 2','Qualificação sócio 2', 'Tipo qualificação sócio 2','Nome do sócio 3', 'Qualificação sócio 3', 'Tipo qualificação sócio 3','Nome do sócio 4', 'Qualificação sócio 4', 'Tipo qualificação sócio 4', 'Nome do sócio 5', 'Qualificação sócio 5', 'Tipo qualificação sócio 5', 'Telefone 1', 'Telefone 2', 'Telefone 3', 'Telefone 4', 'Telefone 5'])

#Mostra a quantidade de CNPJs que não foram enriquecidos
qtd_nulos = df1['Código CNPJ'].isnull().sum()
print('Tivemos {} CNPJs que não foram enriquecidos' .format(qtd_nulos))

#Ordena as colunas de A-Z;
df1.sort_values(by='Razão social', axis=0, inplace=True)

#Remove valores duplicados
df1.duplicated(subset='Código CNPJ', keep='first')

#Tratamento das colunas para inclusão nos Ranges
df1['Capital Social até'] = df1['Capital Social até'].apply(lambda x: str(x).replace('.',''))
df1['Capital Social até'] = df1['Capital Social até'].apply(lambda x: str(x).replace(',','.'))
df1['Capital Social até'] = df1['Capital Social até'].astype('float64')

df1['Faturamento até'] = df1['Faturamento até'].apply(lambda x: str(x).replace('.',''))
df1['Faturamento até'] = df1['Faturamento até'].apply(lambda x: str(x).replace(',','.'))
df1['Faturamento até'] = df1['Faturamento até'].astype('float64')

#Substitui valores nulos pela média de recorrência;
df1['Capital Social até'].fillna(df1['Capital Social até'].mean(), inplace=True)
df1['Faturamento até'].fillna(df1['Faturamento até'].mean(), inplace=True)
df1['Quantidade de funcionários até'].fillna(df1['Quantidade de funcionários até'].mean(), inplace=True)
df1['Porte'].fillna('PEQUENO', inplace=True)

#Cria as variáveis de RANGE monetárias
class_monet = [0,50000,250000,500000,2500000,10000000,50000000,100000000,250000000,500000000,1000000000,1000000000000]
labels_monet = ['R$ 0 - R$ 50 K','R$ 51 K - R$ 250 K','R$ 251 K - R$ 500 K','R$ 501 K - R$ 2.5 M','R$ 2.5 M - R$ 10 M','R$ 10 M - R$ 50 M','R$ 50 M - R$ 100 M','R$ 100 M - R$ 250 MI','R$ 250 M - R$ 500 MI','R$ 500 M - R$ 1 B','+ R$ 1 B']

#Cria a VARIAVEL de Range - Capital Social
RangeCapSocial = pd.cut(x=df1['Capital Social até'], bins=class_monet, labels=labels_monet)

#Cria a VARIAVEL de Range - Faturamento
RangeFat = pd.cut(x=df1['Faturamento até'], bins=class_monet, labels=labels_monet)


#Cria as variáveis de RANGE funcionários
class_func = [0,3,10,50,200,500,1000,5000,10000,100000]
labels_func = ['0 - 3','4 - 10','11 - 50','51 - 200','201 - 500','501 - 1.000','1.001 - 5.000','5.001 - 10.000','+ 10.000']
RangeFunc = pd.cut(x=df1['Quantidade de funcionários até'], bins=class_func, labels=labels_func)


#Atribui as VARs de Ranges nas colunas do dataframe
df1['Faixa de Capital Social'] = RangeCapSocial
df1['Faixa de Faturamento'] = RangeFat
df1['Faixa de Funcionários'] = RangeFunc


df_dados_cad = df1.filter(items=['Código CNPJ','Razão social','Nome fantasia','Logradouro', 'Numero','Complemento', 'Bairro', 'Cidade', 'UF', 'CEP', 'Porte', 'Faixa de Capital Social', 'Faixa de Faturamento', 'Faixa de Funcionários', 'Data da abertura','Tipo de unidade', ' Tipo situação', 'Tipo Enquadramento Porte', 'Código atividade econômica primária', 'Nome atividade econômica primária', 'Código da natureza jurídica', 'Nome natureza jurídica geral', 'E-mail na Receita Federal','Tipo de classificação jurídica', 'Website1', 'Website2', 'Website3', 'Website4', 'Website5', 'Telefone 1', 'Telefone 2', 'Telefone 3', 'Telefone 4', 'Telefone 5'])


df_dados_cad.replace(nan,'', inplace=True)

df_dados_cad['Telefone 1'] = df_dados_cad['Telefone 1'].apply(lambda x: str(x).replace('(','+55').replace(')','').replace(' ','').replace('-',''))
df_dados_cad['Telefone 2'] = df_dados_cad['Telefone 2'].apply(lambda x: str(x).replace('(','+55').replace(')','').replace(' ','').replace('-',''))
df_dados_cad['Telefone 3'] = df_dados_cad['Telefone 3'].apply(lambda x: str(x).replace('(','+55').replace(')','').replace(' ','').replace('-',''))
df_dados_cad['Telefone 4'] = df_dados_cad['Telefone 4'].apply(lambda x: str(x).replace('(','+55').replace(')','').replace(' ','').replace('-',''))
df_dados_cad['Telefone 5'] = df_dados_cad['Telefone 5'].apply(lambda x: str(x).replace('(','+55').replace(')','').replace(' ','').replace('-',''))

df_dados_cad['Website1'] = df_dados_cad['Website1'].apply(lambda x: str(x).replace(',','.'))
df_dados_cad['Website2'] = df_dados_cad['Website2'].apply(lambda x: str(x).replace(',','.'))
df_dados_cad['Website3'] = df_dados_cad['Website3'].apply(lambda x: str(x).replace(',','.'))
df_dados_cad['Website4'] = df_dados_cad['Website4'].apply(lambda x: str(x).replace(',','.'))
df_dados_cad['Website5'] = df_dados_cad['Website5'].apply(lambda x: str(x).replace(',','.'))

df_dados_cad.rename(columns={'Código CNPJ':'CNPJ'}, inplace=True)
df_dados_cad.columns = map(str.upper, df_dados_cad.columns)


# --------------------------------------------------------------------------------- #
# --------------------------Tratamento - Dados de Sócios--------------------------- #
# --------------------------------------------------------------------------------- #

#Separa os sócios em DataFrames;
dfsocios1 = df1.filter(items=['Código CNPJ','Razão social','Nome do sócio 1','Qualificação sócio 1','Tipo qualificação sócio 1'])
dfsocios2 = df1.filter(items=['Código CNPJ','Razão social','Nome do sócio 2','Qualificação sócio 2','Tipo qualificação sócio 2'])
dfsocios3 = df1.filter(items=['Código CNPJ','Razão social','Nome do sócio 3','Qualificação sócio 3','Tipo qualificação sócio 3'])
dfsocios4 = df1.filter(items=['Código CNPJ','Razão social','Nome do sócio 4','Qualificação sócio 4','Tipo qualificação sócio 4'])
dfsocios5 = df1.filter(items=['Código CNPJ','Razão social','Nome do sócio 5','Qualificação sócio 5','Tipo qualificação sócio 5'])

#Renomeia as colunas para padronizar e concatenar;
dfsocios1.rename(columns={'Código CNPJ':'CNPJ','Razão social':'RAZÃO SOCIAL','Nome do sócio 1':'NOME DO SÓCIO','Qualificação sócio 1':'QUALIFICAÇÃO SÓCIO', 'Tipo qualificação sócio 1':'TIPO QUALIFICAÇÃO SÓCIO'}, inplace=True)
dfsocios2.rename(columns={'Código CNPJ':'CNPJ','Razão social':'RAZÃO SOCIAL','Nome do sócio 2':'NOME DO SÓCIO','Qualificação sócio 2':'QUALIFICAÇÃO SÓCIO', 'Tipo qualificação sócio 2':'TIPO QUALIFICAÇÃO SÓCIO'}, inplace=True)
dfsocios3.rename(columns={'Código CNPJ':'CNPJ','Razão social':'RAZÃO SOCIAL','Nome do sócio 3':'NOME DO SÓCIO','Qualificação sócio 3':'QUALIFICAÇÃO SÓCIO', 'Tipo qualificação sócio 3':'TIPO QUALIFICAÇÃO SÓCIO'}, inplace=True)
dfsocios4.rename(columns={'Código CNPJ':'CNPJ','Razão social':'RAZÃO SOCIAL','Nome do sócio 4':'NOME DO SÓCIO','Qualificação sócio 4':'QUALIFICAÇÃO SÓCIO', 'Tipo qualificação sócio 4':'TIPO QUALIFICAÇÃO SÓCIO'}, inplace=True)
dfsocios5.rename(columns={'Código CNPJ':'CNPJ','Razão social':'RAZÃO SOCIAL','Nome do sócio 5':'NOME DO SÓCIO','Qualificação sócio 5':'QUALIFICAÇÃO SÓCIO', 'Tipo qualificação sócio 5':'TIPO QUALIFICAÇÃO SÓCIO'}, inplace=True)

#Concatena os dataframes num só;
df_socios = pd.concat([dfsocios1,dfsocios2,dfsocios3,dfsocios4,dfsocios5])

df_socios.dropna(subset=['NOME DO SÓCIO'], inplace=True)


# --------------------------------------------------------------------------------- #

#Variaveis de OUTPUT;
#output_file_path = 'C:\\Projetos\\Python\\Data Desktop\\Exportacao\\'
#arquivo_output = 'output_data_desktop' + '.xlsx'

#Cria um excel writer usando xlsxwriter como engine;
#writer = pd.ExcelWriter(output_file_path + arquivo_output, engine='xlsxwriter')

#Escreve os dataframes em diferentes sheets dentro do arquivo final;
#df_dados_cad.to_excel(writer, sheet_name='Dados Cadastrais - DB - Cortex', index=False)
#df_socios.to_excel(writer, sheet_name='Sócios - DB - Cortex', index=False)

#Fecha o arquivo e salva no diretório;
#writer.save()
print('Lista exportada com sucesso! =)')


Tivemos 154 CNPJs que não foram enriquecidos
Lista exportada com sucesso! =)
